<center>
<h1 style="font-family:verdana">
 💻 🧑 Classificació d'intencions 🧑 💻


<p> 🎯 <b>Objectiu</b>: en aquesta pràctica aprendrem a detectar la intenció de l'usuari a partir d'interaccions reals amb un xatbot. En el context de xatbots, la classificació d'intencions ajuda a entendre quina acció o resposta hauria de prendre el sistema en funció de la consulta de l'usuari.  


<p> ✨ <b>Contingut</b>: en primer lloc, farem servir una base de dades amb oracions d'interaccions en espanyol etiquetades com 19 intencions diferents. En segon lloc, realitzarem el preprocessament de les dades, és a dir, transformarem les dades perquè tinguen un format adequat per a ser introduïdes al model. I finalment, dissenyarem i entrenarem el model de classificació per detectar automàticament la intenció de les oracions.</p>  


<p> ✏ <b>Exercicis</b>: en cada secció anireu trobant exercicis que haureu d'anar resolent. </p>



---

<h2> Índex </h2>

1. [Inspecció del conjunt de dades](#section-one)
  * [Exercici 1](#ex-one)
2. [Preprocessament de dades](#section-two)
  * [Exercici 2](#ex-two)
  * [Exercici 3](#ex-three)
3. [Disseny del model i entrenament](#section-three)
  * [Exercici 4](#ex-four)
  * [Exercici 5](#ex-five)
4. [Lliurable](#section-four)
---

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D, Dropout, Conv1D, GlobalAveragePooling1D, LayerNormalization
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import GlobalAveragePooling1D # Import the layer here

In [9]:
!pip install gdown
!gdown "https://drive.google.com/uc?id=1u2wzXvsuscLeFHwXcDwMDaNDy0u_99-t"
!tar -zxf nlu_ATIS_data.tar.gz

^C



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Usuario\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip
Downloading...
From: https://drive.google.com/uc?id=1u2wzXvsuscLeFHwXcDwMDaNDy0u_99-t
To: c:\Users\Usuario\OneDrive\Documents\GRAU IA\Tercer\TVD\LAB\Voice-and-Dialogue-Treatment\Lab2\nlu_ATIS_data.tar.gz

  0%|          | 0.00/122k [00:00<?, ?B/s]
100%|██████████| 122k/122k [00:00<00:00, 88.0MB/s]


<h1><a name="section-one"> 1. Inspecció del conjunt de dades </a></h1>

A la carpeta `data` tenim els diferents fitxers CSV que utilitzarem per a aquesta pràctica.

En primer lloc, llegirem les dades dels fitxers CSV amb `pandas`.

In [10]:
train_data = pd.read_csv('./data/train.csv', header=None)
val_data = train_data.tail(900)
train_data = pd.read_csv('./data/train.csv', header=None, nrows=4078)
test_data = pd.read_csv('./data/test.csv', header=None)

print('Training size:', len(train_data))
print('Validation dataset size:', len(val_data))
print('Test dataset size:', len(test_data))

Training size: 4078
Validation dataset size: 900
Test dataset size: 893


Per a aquesta primera part de la pràctica ens centrarem en la primera columna dels arxius que correspon amb les **oracions** en anglès introduïdes per l'usuari. I en la tercera columna que correspon amb la **intenció** de cada oració, és a dir, cada oració tindrà una etiqueta.

Podeu executar la cel·la següent tantes vegades com vulgueu per veure instàncies d'aquest conjunt de dades.


In [11]:
random_number = random.randint(0, len(train_data)-1)

train_sentences = list(train_data[0])
train_labels = list(s.replace('"', '') for s in train_data[2])
train_labels = list(s.replace(' ', '') for s in train_labels)

print('Sentence: ', train_sentences[random_number])
print('Intent: ', train_labels[random_number])

Sentence:  what 's the fare for delta flight 217 from dallas to san francisco
Intent:  airfare


A continuació analitzarem quantes etiquetes diferents hi ha al dataset i quines són.

In [5]:
num_labels = 0
for label in set(train_labels):
  print(f'Label {num_labels}:', label.split('.')[-1])
  num_labels += 1

print(f'\nThere are a total of {num_labels} intent labels')

Label 0: ground_service+ground_fare
Label 1: ground_service
Label 2: cheapest
Label 3: abbreviation
Label 4: flight_no
Label 5: distance
Label 6: airport
Label 7: airline
Label 8: flight
Label 9: ground_fare
Label 10: airfare
Label 11: aircraft
Label 12: aircraft+flight+flight_no
Label 13: quantity
Label 14: capacity
Label 15: flight+airfare
Label 16: restriction
Label 17: airline+flight_no
Label 18: flight_time
Label 19: city
Label 20: meal
Label 21: airfare+flight_time

There are a total of 22 intent labels


<h1><a name="section-two"> 2. Preprocessament de dades </a></h1>

En primer lloc, haurem de tokenitzar les oracions. Això consisteix a convertir el text en representacions numèriques, ja que els models esperen unitats discretes.

En aquesta pràctica farem servir una tokenització senzilla, simplement dividirem les oracions en paraules i crearem un vocabulari basat en les paraules úniques de les dades d'entrenament. Cada paraula (token) tindrà assignat un ID únic.

Vegem com queda el vocabulari.

In [14]:
num_words=500 
tokenizer = Tokenizer(num_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences)

vocab = tokenizer.word_index
print(vocab)
print(len(vocab))

{'<OOV>': 1, 'to': 2, 'from': 3, 'flights': 4, 'the': 5, 'on': 6, 'what': 7, 'me': 8, 'flight': 9, 'boston': 10, 'show': 11, 'san': 12, 'i': 13, 'denver': 14, 'a': 15, 'francisco': 16, 'in': 17, 'and': 18, 'atlanta': 19, 'pittsburgh': 20, 'is': 21, 'dallas': 22, 'baltimore': 23, 'all': 24, 'philadelphia': 25, 'like': 26, 'are': 27, 'list': 28, 'airlines': 29, 'of': 30, 'between': 31, 'that': 32, 'washington': 33, 'leaving': 34, 'please': 35, 'pm': 36, 'morning': 37, 'would': 38, 'fly': 39, 'for': 40, 'fare': 41, 'first': 42, 'wednesday': 43, 'after': 44, 'there': 45, 'oakland': 46, "'d": 47, 'ground': 48, 'you': 49, 'does': 50, 'trip': 51, 'transportation': 52, 'class': 53, 'arriving': 54, 'cheapest': 55, 'need': 56, 'city': 57, 'round': 58, 'with': 59, 'before': 60, 'which': 61, 'available': 62, 'have': 63, 'give': 64, 'at': 65, 'fares': 66, 'american': 67, 'afternoon': 68, 'one': 69, 'want': 70, 'how': 71, 'way': 72, 'new': 73, 'dc': 74, 'nonstop': 75, 'arrive': 76, 'earliest': 77, '

---

 <h1><a name="ex-one"><center> ✏ Exercici 1 ✏</a></h1>

En aquest primer exercici us demanem que donat el vocabulari anterior convertiu la llista d'oracions de la partició d'entrenament, és a dir, `train_sentenes` en seqüències d'IDs.

Podeu trobar la documentació [aquí](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer).

In [15]:
#TODO
print(train_sentences[0])

train_sequences = tokenizer.texts_to_sequences(train_sentences)


print(train_sequences[0])

i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
[13, 70, 2, 39, 3, 10, 65, 416, 85, 18, 76, 17, 14, 65, 494, 17, 5, 37]


Si ho heu fet correctament hauríeu d'obtenir això:

```
print(train_sentences[0])
print(train_sequences[0])

i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
[12, 69, 1, 38, 2, 9, 64, 415, 84, 17, 75, 16, 13, 64, 493, 16, 4, 36]
```



---
A continuació haurem d'aconseguir que totes les seqüències tinguen una longitud fixa. Per a fer això primer fixarem la longitud segons la longitud màxima trobada a les seqüències del conjunt d'entrenament. I a continuació omplirem (*pad*) les seqüències que tinguen una longitud menor.


In [16]:
max_sequence_length = max(map(len, train_sequences))
train_pad_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length)
print('Padded sequence: ', train_pad_sequences[0])

Padded sequence:  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  13  70   2  39   3  10  65 416
  85  18  76  17  14  65 494  17   5  37]


---

 <h1><a name="ex-two"><center> ✏ Exercici 2 ✏</a></h1>

Com l'ordre de les paraules sí que importa als models que utilitzarem en aquesta pràctica, és aconsellable que el *padding* estiga al final i no al principi. Busqueu [aquí](https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences) com fer perquè el codi anterior afegisca els zeros al final i no al principi de la seqüència.

In [17]:
#TODO
# Apliquem el padding='post'
train_pad_sequences = pad_sequences(train_sequences, padding='post', maxlen=100)
print('Padded sequence: ', train_pad_sequences[0])

Padded sequence:  [ 13  70   2  39   3  10  65 416  85  18  76  17  14  65 494  17   5  37
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


Si ho heu fet correctament hauríeu d'obtenir això:

```
print('Padded sequence: ', train_pad_sequences[0])

Padded sequence:  [ 12  69   1  38   2   9  64 415  84  17  75  16  13  64 493  16   4  36
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0]
```


---

A continuació convertirem les classes d'intencions categòriques (*capacity*, *ground_service*, *flight*, etc.) en el que anomenem one-hot vector encoding. Aquesta tècnica s'utilitza per representar les dades categòriques com a vectors binaris. On cada vector representa una classe específica i l'element corresponent a la classe es posa a 1 i la resta d'elements es mantenen a 0.

Imaginem que tenim tres classes: *capacity*, *ground_service*, *flight*. Podríem codificar aquestes classes amb un vector únic de la forma següent:


```
   capacity -> [1, 0, 0]
   ground_service -> [0, 1, 0]
   flight -> [0, 0, 1]
```

Per aconseguir això primer codificarem les classes d'intenció en etiquetes numèriques.

In [18]:
label_encoder = LabelEncoder()
train_numerical_labels = label_encoder.fit_transform(train_labels)

print(f'Original labels: {train_labels}\n')
print(f'Encoded labels: {train_numerical_labels} \n')

Original labels: ['flight', 'flight', 'flight_time', 'airfare', 'airfare', 'flight', 'aircraft', 'flight', 'flight', 'ground_service', 'flight', 'flight', 'airport', 'flight', 'flight', 'airfare', 'ground_service', 'flight', 'flight', 'flight', 'flight', 'flight', 'flight', 'aircraft', 'airfare', 'flight', 'airline', 'flight', 'ground_service', 'flight', 'airfare', 'flight', 'flight', 'flight', 'flight', 'airfare', 'airline', 'flight', 'flight', 'flight', 'distance', 'flight', 'airline', 'airline', 'flight', 'airline', 'ground_service', 'abbreviation', 'flight', 'flight', 'flight_time', 'flight', 'flight', 'ground_fare', 'flight', 'abbreviation', 'flight', 'flight', 'flight', 'flight', 'flight', 'airline', 'flight', 'ground_service', 'airline', 'flight', 'flight', 'airport', 'flight', 'flight', 'abbreviation', 'flight', 'flight', 'flight', 'flight', 'aircraft', 'airfare', 'flight', 'flight', 'flight', 'flight', 'flight', 'flight', 'flight', 'airline', 'flight', 'flight', 'flight', 'fli

I a continuació convertim les etiquetes a vectors one-hot.

In [19]:
num_classes = len(np.unique(train_numerical_labels))
train_encoded_labels = to_categorical(train_numerical_labels, num_classes)

print('Example: \n')
print(f'Original label: {train_labels[0]}\n')
print(f'Numerical label: {train_numerical_labels[0]}\n')
print(f'One-hot: {train_encoded_labels[0]}\n')

Example: 

Original label: flight

Numerical label: 12

One-hot: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



---

 <h1><a name="ex-three"><center> ✏ Exercici 3 ✏</a></h1>

Amb la partició de validació i test haurem de realitzar els mateixos passos. Per tant, en aquest exercici us demanem que obtingueu `val_pad_sequences`, `val_encoded_labels`, `test_pad_sequences` i `test_encoded_labels`.

In [20]:
#TODO
val_sentences = list(val_data[0])
val_sequences = tokenizer.texts_to_sequences(val_sentences)
val_pad_sequences = pad_sequences(val_sequences, padding='post', maxlen=100)
print(len(val_pad_sequences), len(val_sequences))

test_sentences = list(test_data[0])
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_pad_sequences = pad_sequences(test_sequences, padding='post', maxlen=100)
print(len(test_pad_sequences), len(test_sequences))


900 900
893 893


In [21]:
val_labels = list(val_data[2])
val_labels = list(s.replace('"', '') for s in val_labels)
val_labels = list(s.replace(' ', '') for s in val_labels)

test_labels = list(test_data[2])
test_labels = list(s.replace('"', '') for s in test_labels)
test_labels = list(s.replace(' ', '') for s in test_labels)

def remove_values_and_indices(input_list, values_to_remove, other_list):
    #print(f'Initial {len(input_list)} samples')
    #print(f'Initial {len(other_list)} samples')
    indices_to_remove = [idx for idx, item in enumerate(input_list) if item in values_to_remove]
    #print(f'Removing {len(indices_to_remove)} samples')
    cleaned_list = [item for item in input_list if item not in values_to_remove]
    #print(f'Remaining {len(cleaned_list)} samples')
    cleaned_other_list = [item for idx, item in enumerate(other_list) if idx not in indices_to_remove]
    #print(f'Remaining {len(cleaned_other_list)} samples')
    return cleaned_list, np.array(cleaned_other_list)

values_to_remove = ['day_name','airfare+flight','flight+airline','flight_no+airline', 'flight']
val_labels, val_pad_sequences = remove_values_and_indices(val_labels, values_to_remove, val_pad_sequences)
test_labels, test_pad_sequences = remove_values_and_indices(test_labels, values_to_remove, test_pad_sequences)

In [22]:
val_encoded_labels = to_categorical(label_encoder.transform(val_labels), num_classes)
test_encoded_labels = to_categorical(label_encoder.transform(test_labels), num_classes)
print('Validation labels shape:', val_encoded_labels.shape)
print('Test labels shape:', test_encoded_labels.shape)

Validation labels shape: (257, 22)
Test labels shape: (256, 22)


---

<h1><a name="section-three"> 3. Disseny del model i entrenament </a></h1>

En primer lloc, anem a comprovar si hi ha GPUs disponibles. A continuació si hi ha GPUs disponibles el codi assegurarà que *TensorFlow* només assigne memòria GPU quan siga necessari.

In [2]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available!")
else:
    print("GPU is not available. The model will be trained on CPU.")

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

GPU is not available. The model will be trained on CPU.


---

 <h1><a name="ex-four"><center> ✏ Exercici 4 ✏</a></h1>

En aquest exercici haureu de dissenyar l'arquitectura del model. El nostre model tindrà quatre capes:

1. La primera capa serà un **embedding**. Aquesta capa permetrà convertir les dades de text d'entrada, en vectors densos amb una mida fixa (*embedding_dim*). Aquesta representació més compacta permetrà per una part capturar la informació semàntica del text d'entrada, permetent així generalitzar millor i comprendre les relacions entre les paraules. I, per una altra banda, reduir la complexitat computacional, accelerant així el temps d'entrenament i inferència. En resum, aquesta capa assignarà a cada índex de cada paraula un vector dens de mida *embedding_dim*.

2. La segona capa serà un **pooling** layer. L'entrada d'aquesta capa serà un tensor 3D (*batch_size*, *sequence_length*, *embedding_dim*). Aquesta capa es centrarà a capturar la informació més important de la seqüència d'entrada, és a dir, prendrà el valor màxim de la seqüència, donant lloc a un tensor 2D (batch_size, embedding_dim).

3. La tercera capa serà una capa **densa**. És a dir, una capa completament connectada (*fully-connected*): cada neurona d'aquesta capa estarà connectada a totes les neurones de la capa anterior. La funció d'activació que utilitzarem serà una ReLU. Aquesta funció introdueix una no-linealitat al model permetent així aprendre relacions complexes en les dades.

4. L'última capa també serà una capa **densa**. En aquest cas la funció d'activació haurà de ser la funció Softmax. Aquesta funció es fa servir per a convertir els valors de la capa anterior (*logits*) en probabilitats normalitzades. El valor de cada element de sortida representarà la probabilitat que l'entrada pertanya a una classe específica.


📢  Les capes que haureu de fer servir les podreu trobar [aquí](https://www.tensorflow.org/api_docs/python/tf/keras/layers).





Veiem que la mesura d'***Accuracy*** és un mal indicador en el qual basar l'entrenament, perquè les classes que tenim són molt desbalancejades. Una mètrica que ens pot ser més útil és, per exemple, la ***F1-score*** de Keras.

In [4]:
# Importar métricas
from tensorflow.keras.metrics import F1Score

# Modelo
num_words = 300
embedding_dim = 256
vocab_size = num_words + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))  # layer 1
model.add(GlobalMaxPooling1D(data_format='channels_last'))  # layer 2
model.add(Dense(128, activation='relu'))  # layer 3
model.add(Dense(num_classes, activation='softmax'))  # layer 4

metrics = tf.keras.metrics.F1Score(average='macro')
# Compile con métricas básicas
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = [metrics]
)

# Train
batch_size = 32
epochs = 10
model.fit(
    train_pad_sequences,
    train_encoded_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_pad_sequences, val_encoded_labels)
)

# Evaluation (Keras devuelve las métricas indicadas)
results = model.evaluate(test_pad_sequences, test_encoded_labels, batch_size=batch_size, return_dict=True)
print(f'Test Loss: {results["loss"]:.4f}')
print(f'Test F1 Score: {results["f1_score"]:.4f}')

NameError: name 'max_sequence_length' is not defined

Podeu veure a continuació les oracions que el model ha classificat incorrectament.

In [25]:
probs = model.predict(test_pad_sequences)
_predicted_labels = np.argmax(probs, axis=1)
predicted_labels = label_encoder.inverse_transform(_predicted_labels)

for i in range(0, len(predicted_labels)):
  if test_labels[i] != predicted_labels[i]:
    print('Sentence: ', test_sentences[i])
    print('Original label: ', test_labels[i])
    print('Predicted label: ', predicted_labels[i])
    print()


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Sentence:  on april first i need a ticket from tacoma to san jose departing before 7 am
Original label:  flight+airfare
Predicted label:  flight

Sentence:  i would like a flight traveling one way from phoenix to san diego on april first
Original label:  meal
Predicted label:  flight

Sentence:  i would like a flight from orlando to salt lake city for april first on delta airlines
Original label:  meal
Predicted label:  flight

Sentence:  i need a flight from toronto to newark one way leaving wednesday evening or thursday morning
Original label:  airport
Predicted label:  ground_service

Sentence:  on wednesday april sixth i would like to fly from long beach to columbus after 3 pm
Original label:  airline
Predicted label:  flight

Sentence:  are there any flights from long beach to columbus on wednesday april sixth
Original label:  city
Predicted label:  flight

Sentence:  show flight and prices kansas city to chicago on next wednesday arriving in 

---

 <h1><a name="ex-five"><center> ✏ Exercici 5 ✏ </a></h1>

Modifiqueu els següents paràmetres del model anterior i analitzeu com afecten a la seva *accuracy*:

 1. **Preprocessament.** Modifiqueu el Tokenizer per canviar la mida del vostre vocabulari i afegiu nous passos de preprocessament. Alguns possibles canvis són canviar la mida del vocabulari, treure la capitalització o fer servir *lemmatització* o *stemming*.

 2. **Mida dels Embeddings.** Proveu diferents mides d'*Embeddings* i observeu com canvia l'*accuracy* del model. Heu d'explicar les vostres conclusions.

 3. **Xarxes Convolucionals.** Afegiu capes convolucionals al vostre model. Expliqueu amb detall els valors que heu provat i la vostra motivació a l'hora d'escollir-los. Recordeu, que també podeu provar diferents configuracions de *pooling*.

 4. **Xarxes Recurrents.**  Afegiu capes recurrents al vostre model (LSTM, GRU). Expliqueu amb detall els valors que heu provat i la vostra motivació.

 5. **Regularització.** Quan proveu configuracions amb més paràmetres veureu que el model comença a tenir *overfitting* molt prompte durant l'entrenament. Afegiu *Dropout* al vostre model. Heu d'explicar la vostra decisió de valors i de posició dins de la xarxa.

 6. **Balancejat de les classes.** Si analitzeu el dataset, veureu que la freqüència de les classes està molt desbalancejada. Keras us permet afegir un pes per a cada classe a l'hora de calcular la loss (Mireu el paràmetre "class_weigth" a la documentació https://keras.io/api/models/model_training_apis/). Calculeu un pes per a cada classe i afegiu-lo al mètode fit del vostre model.

 ---

***Early stopping*** (basat en validation) ens permet parar abans de fer *overfitting*, és una bona forma de determinar el nombre de *epochs*. Les epochs les hem de controlar en cada xarxa nova.  

Anem provant respecte les anteriors. Podem provar LSTM (mínim), i la combinació amb la convolucional (segurament és la millor opció). També es pot regular amb *batch normalization*.

**1. PREPROCESSAMENT**

Experimantarem amb canviar la mida del vocabulari,

In [103]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt

# Descargar recursos necesarios de NLTK (ejecutar solo una vez)
try:
    nltk.data.find('corpora/stopwords')
    nltk.data.find('corpora/wordnet')
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('stopwords')
    nltk.download('wordnet')
    nltk.download('punkt')
    nltk.download('omw-1.4')

# Inicializar herramientas de preprocesamiento
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Función de preprocesamiento
def preprocess_text(text, lowercase=True, remove_stopwords=False, use_lemmatization=False):
    """
    Preprocesa el texto aplicando diferentes técnicas.
    
    Args:
        text: Texto a preprocesar
        lowercase: Convertir a minúsculas
        remove_stopwords: Eliminar stopwords
        use_lemmatization: Aplicar lemmatización
    
    Returns:
        Texto preprocesado
    """
    # Convertir a minúsculas
    if lowercase:
        text = text.lower()
    
    # Tokenizar
    words = text.split()
    
    # Eliminar stopwords
    if remove_stopwords:
        words = [word for word in words if word not in stop_words]
    
    # Aplicar lemmatización
    if use_lemmatization:
        words = [lemmatizer.lemmatize(word) for word in words]
    
    return ' '.join(words)

# APLICAREM:
# Minúscules
# Stopwords
# Lematització

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


**2. MIDA DELS EMBEDDINGS**

Provarem de canviar la mida dels *embeddings* i observarem el seu efecte en l'*accuracy* del model resultant.

In [ ]:
def preprocess_intent_recognition(train_data: pd.DataFrame, val_data: pd.DataFrame, test_data: pd.DataFrame, num_words: int = None) -> Dict[str, np.ndarray|int]:

    # Training data
    train_sentences = list(train_data[0])
    train_labels = list(s.replace('"', '') for s in train_data[2])
    train_labels = list(s.replace(' ', '') for s in train_labels)
    # Validation data
    val_sentences = list(val_data[0])
    val_labels = list(val_data[2])
    val_labels = list(s.replace('"', '') for s in val_labels)
    val_labels = list(s.replace(' ', '') for s in val_labels)
    # Test data
    test_sentences = list(test_data[0])
    test_labels = list(test_data[2])
    test_labels = list(s.replace('"', '') for s in test_labels)
    test_labels = list(s.replace(' ', '') for s in test_labels)

    ######## Tokenization and Padding of sentences ########    

    train_sentences = [preprocess_text(s, lowercase=True, use_lemmatization=True, remove_stopwords=True) for s in train_sentences]
    val_sentences = [preprocess_text(s, lowercase=True, use_lemmatization=True, remove_stopwords=True) for s in val_sentences]
    test_sentences = [preprocess_text(s, lowercase=True, use_lemmatization=True, remove_stopwords=True) for s in test_sentences]
    
    # Limitar a 300 voacb size (NOU) -> Si dona error escric!
    # f1_score de skit learn (Hui)
    # 256 embedding dim
    # Combinacions dels meus
    tokenizer = Tokenizer(num_words=300, oov_token="<OOV>")
    tokenizer.fit_on_texts(train_sentences)

    # Train data
    train_sequences = tokenizer.texts_to_sequences(train_sentences)
    maxlen = max(map(len, train_sequences))
    train_pad_sequences = pad_sequences(train_sequences, padding='post', maxlen=maxlen)
    # Validation data
    val_sequences = tokenizer.texts_to_sequences(val_sentences)
    val_pad_sequences = pad_sequences(val_sequences, padding='post', maxlen=maxlen)
    # Test data
    test_sequences = tokenizer.texts_to_sequences(test_sentences)
    test_pad_sequences = pad_sequences(test_sequences, padding='post', maxlen=maxlen)

    ######## Label Encoding and One-Hot Encoding of labels ########

    label_encoder = LabelEncoder()
    # Training data
    train_numerical_labels = label_encoder.fit_transform(train_labels)
    num_classes = len(np.unique(train_numerical_labels))
    train_encoded_labels = to_categorical(train_numerical_labels, num_classes)
    # Aux function to remove unseen labels and corresponding sequences in val and test data
    values_to_remove = ['day_name','airfare+flight','flight+airline','flight_no+airline', 'flight']
    def remove_values_and_indices(input_list, values_to_remove, other_list):
        indices_to_remove = [idx for idx, item in enumerate(input_list) if item in values_to_remove]
        cleaned_list = [item for item in input_list if item not in values_to_remove]
        cleaned_other_list = [item for idx, item in enumerate(other_list) if idx not in indices_to_remove]
        return cleaned_list, np.array(cleaned_other_list)
    # Validation data
    val_labels, val_pad_sequences = remove_values_and_indices(val_labels, values_to_remove, val_pad_sequences)
    val_encoded_labels = to_categorical(label_encoder.transform(val_labels), num_classes)
    # Test data
    test_labels, test_pad_sequences = remove_values_and_indices(test_labels, values_to_remove, test_pad_sequences)
    test_encoded_labels = to_categorical(label_encoder.transform(test_labels), num_classes)

    # Calcula vocab_size correctamente
    vocab_size = max(tokenizer.word_index.values()) + 1
    
    ##### Return preprocessed data and metadata ########
    data = {
        'train_X': train_pad_sequences,
        'train_y': train_encoded_labels,
        'val_X': val_pad_sequences,
        'val_y': val_encoded_labels,
        'test_X': test_pad_sequences,
        'test_y': test_encoded_labels,
        'num_classes': num_classes,
        'maxlen': maxlen,
        'vocab_size': vocab_size
    }
    return data

In [115]:
train_data = pd.read_csv('./data/train.csv', header=None)
val_data = train_data.tail(900)
train_data = pd.read_csv('./data/train.csv', header=None, nrows=4078)
test_data = pd.read_csv('./data/test.csv', header=None)

data = preprocess_intent_recognition(train_data, val_data, test_data)

train_pad_sequences = data['train_X']
train_encoded_labels = data['train_y']
val_pad_sequences = data['val_X']
val_encoded_labels = data['val_y']
test_pad_sequences = data['test_X']
test_encoded_labels = data['test_y']
num_classes = data['num_classes']
max_sequence_length = data['maxlen']
vocab_size = data['vocab_size']

In [116]:
embedding_dim = 256

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

metrics = tf.keras.metrics.F1Score(average='macro')
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[metrics]
)

# Entrena
batch_size = 32
epochs = 10
model.fit(
    train_pad_sequences,
    train_encoded_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_pad_sequences, val_encoded_labels)
)

# Evalúa
results = model.evaluate(test_pad_sequences, test_encoded_labels, batch_size=batch_size, return_dict=True)
print(f'Test Loss: {results["loss"]:.4f}')
print(f'Test F1 Score: {results["f1_score"]:.4f}')

Epoch 1/10


c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - f1_score: 0.0657 - loss: 1.4683 - val_f1_score: 0.1428 - val_loss: 1.4601
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.2625 - loss: 0.3408 - val_f1_score: 0.2206 - val_loss: 0.9513
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.3527 - loss: 0.1653 - val_f1_score: 0.3711 - val_loss: 0.6226
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.4911 - loss: 0.0972 - val_f1_score: 0.4954 - val_loss: 0.6158
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.6177 - loss: 0.0578 - val_f1_score: 0.5019 - val_loss: 0.7379
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - f1_score: 0.6508 - loss: 0.0370 - val_f1_score: 0.5277 - val_loss: 0.6419
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.6533 - loss: 0.0281 - val_f1_score: 0.5139 - val_loss: 0.7013
Epoch 8/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.6434 - loss: 0.0250 - val_f1_score: 0.5326 - val_

**3. XARXES CONVOLUCIONALS**

En aquest apartat, experimentarem amb afegir capes convolucionals al nostre model.

In [122]:
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

In [127]:
def test_model(model=model):
        
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['f1_score'])

    # Train the model
    batch_size = 32
    epochs = 10
    model.fit(train_pad_sequences, train_encoded_labels, batch_size=batch_size, epochs=epochs, validation_data=(val_pad_sequences, val_encoded_labels))

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(test_pad_sequences, test_encoded_labels, batch_size=batch_size)
    print(f"Test accuracy: {accuracy[0]:.2f}")

In [135]:
model1 = Sequential()
model1.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model1.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dense(num_classes, activation='softmax'))

test_model(model1)

Epoch 1/10


c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - f1_score: 0.0694 - loss: 1.3793 - val_f1_score: 0.1503 - val_loss: 1.4054
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - f1_score: 0.2452 - loss: 0.3130 - val_f1_score: 0.2835 - val_loss: 0.8773
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.3962 - loss: 0.1285 - val_f1_score: 0.3702 - val_loss: 0.6844
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.5305 - loss: 0.0660 - val_f1_score: 0.3629 - val_loss: 0.6233
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - f1_score: 0.6093 - loss: 0.0529 - val_f1_score: 0.5027 - val_loss: 0.6262
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.6684 - loss: 0.0389 - val_f1_score: 0.5181 - val_loss: 0.5659
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.7014 - loss: 0.0224 - val_f1_score: 0.5333 - val_loss: 0.6692
Epoch 8/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.6968 - loss: 0.0123 - val_f1_score: 0.5370 - val_

In [136]:
model2 = Sequential()
model2.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model2.add(Conv1D(128, kernel_size=3, activation='relu', padding='same'))
model2.add(Conv1D(128, kernel_size=5, activation='relu', padding='same'))
model2.add(MaxPooling1D(pool_size=2))  # reducció gradual
model2.add(GlobalMaxPooling1D())
model2.add(Dense(128, activation='relu'))
model2.add(Dense(num_classes, activation='softmax'))

test_model(model2)

Epoch 1/10


c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - f1_score: 0.0621 - loss: 1.3724 - val_f1_score: 0.1455 - val_loss: 1.3374
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - f1_score: 0.2156 - loss: 0.3741 - val_f1_score: 0.2354 - val_loss: 0.8145
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - f1_score: 0.3737 - loss: 0.1722 - val_f1_score: 0.3447 - val_loss: 0.6529
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - f1_score: 0.4923 - loss: 0.0954 - val_f1_score: 0.4333 - val_loss: 0.6965
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - f1_score: 0.5582 - loss: 0.0536 - val_f1_score: 0.5288 - val_loss: 0.6308
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - f1_score: 0.6857 - loss: 0.0350 - val_f1_score: 0.5587 - val_loss: 0.5210
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - f1_score: 0.6433 - loss: 0.0287 - val_f1_score: 0.5713 - val_loss: 0.5913
Epoch 8/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - f1_score: 0.7279 - loss: 0.0241 - val_f1_score: 0.5904 - v

In [155]:
configuracions = [
    {
        "filters": 32,
        "kernel_size": 3,
        "pooling": "global_max"
    }
]


In [176]:

filtres = [32, 64, 128, 160, 192]      
kernels = [3, 5, 7, 9]                 
poolings = ['global_max', 'global_avg', 'max_i_global']

configuracions = [
    {"filters": f, "kernel_size": k, "pooling": p}
    for f in filtres
    for k in kernels
    for p in poolings
]


In [158]:
configuracions = []
for f in [32, 64, 128]:
    for k in [3, 5, 7]:
        for p in ["global_max", "global_avg", "max_i_global"]:
            configuracions.append({"filters": f, "kernel_size": k, "pooling": p})

print(configuracions)

[{'filters': 32, 'kernel_size': 3, 'pooling': 'global_max'}, {'filters': 32, 'kernel_size': 3, 'pooling': 'global_avg'}, {'filters': 32, 'kernel_size': 3, 'pooling': 'max_i_global'}, {'filters': 32, 'kernel_size': 5, 'pooling': 'global_max'}, {'filters': 32, 'kernel_size': 5, 'pooling': 'global_avg'}, {'filters': 32, 'kernel_size': 5, 'pooling': 'max_i_global'}, {'filters': 32, 'kernel_size': 7, 'pooling': 'global_max'}, {'filters': 32, 'kernel_size': 7, 'pooling': 'global_avg'}, {'filters': 32, 'kernel_size': 7, 'pooling': 'max_i_global'}, {'filters': 64, 'kernel_size': 3, 'pooling': 'global_max'}, {'filters': 64, 'kernel_size': 3, 'pooling': 'global_avg'}, {'filters': 64, 'kernel_size': 3, 'pooling': 'max_i_global'}, {'filters': 64, 'kernel_size': 5, 'pooling': 'global_max'}, {'filters': 64, 'kernel_size': 5, 'pooling': 'global_avg'}, {'filters': 64, 'kernel_size': 5, 'pooling': 'max_i_global'}, {'filters': 64, 'kernel_size': 7, 'pooling': 'global_max'}, {'filters': 64, 'kernel_size'

In [177]:
def crear_model(config):
    
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
    model.add(Conv1D(config["filters"], kernel_size=config["kernel_size"], activation='relu', padding='same'))
    
    if config["pooling"] == "global_max":
        model.add(GlobalMaxPooling1D())
    elif config["pooling"] == "global_avg":
        model.add(GlobalAveragePooling1D())
    elif config["pooling"] == "max_i_global":
        model.add(MaxPooling1D(pool_size=2))
        model.add(GlobalMaxPooling1D())
    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['f1_score'])
    return model

In [ ]:
results = []

for i, cfg in enumerate(configuracions):
    print("Entrenant model " + str(i+1) + "/" + str(len(configuracions)) + ": " + str(cfg))
    model = crear_model(cfg)

    history = model.fit(
        train_pad_sequences, train_encoded_labels,
        epochs=10, batch_size=32,
        validation_data=(val_pad_sequences, val_encoded_labels),
        verbose=0
    )

    # F1 mitjà per epoch
    train_f1_per_epoch = [np.mean(v) for v in history.history['f1_score']]
    val_f1_per_epoch   = [np.mean(v) for v in history.history['val_f1_score']]

    # Extreure valors resum
    train_f1_last = train_f1_per_epoch[-1]
    val_f1_last   = val_f1_per_epoch[-1]
    val_f1_best   = max(val_f1_per_epoch)
    best_epoch    = np.argmax(val_f1_per_epoch) + 1

    results.append({
        "filters": cfg["filters"],
        "kernel_size": cfg["kernel_size"],
        "pooling": cfg["pooling"],
        "train_f1_last": train_f1_last,
        "val_f1_last": val_f1_last,
        "best_val_f1": val_f1_best,
        "best_epoch": best_epoch
    })

df_results = pd.DataFrame(results)
print("\nResultats finals:")
print(df_results)

Entrenant model 1/60: {'filters': 32, 'kernel_size': 3, 'pooling': 'global_max'}
Entrenant model 2/60: {'filters': 32, 'kernel_size': 3, 'pooling': 'global_avg'}
Entrenant model 3/60: {'filters': 32, 'kernel_size': 3, 'pooling': 'max_i_global'}
Entrenant model 4/60: {'filters': 32, 'kernel_size': 5, 'pooling': 'global_max'}
Entrenant model 5/60: {'filters': 32, 'kernel_size': 5, 'pooling': 'global_avg'}
Entrenant model 6/60: {'filters': 32, 'kernel_size': 5, 'pooling': 'max_i_global'}
Entrenant model 7/60: {'filters': 32, 'kernel_size': 7, 'pooling': 'global_max'}
Entrenant model 8/60: {'filters': 32, 'kernel_size': 7, 'pooling': 'global_avg'}
Entrenant model 9/60: {'filters': 32, 'kernel_size': 7, 'pooling': 'max_i_global'}
Entrenant model 10/60: {'filters': 32, 'kernel_size': 9, 'pooling': 'global_max'}
Entrenant model 11/60: {'filters': 32, 'kernel_size': 9, 'pooling': 'global_avg'}
Entrenant model 12/60: {'filters': 32, 'kernel_size': 9, 'pooling': 'max_i_global'}
Entrenant model 1

In [170]:
import plotly.express as px

fig = px.bar(
    df_results,
    x="kernel_size",
    y="best_val_f1",
    color="pooling",
    barmode="group",
    title="Millor F1 de validació per configuració",
    labels={"best_val_f1": "F1 Score", "kernel_size": "Kernel"}
)
fig.show()


In [172]:
fig = px.scatter(
    df_results,
    x="train_f1_last",
    y="val_f1_last",
    color="pooling",
    size="filters",
    hover_data=["kernel_size"],
    title="Relació Train vs Val F1"
)
fig.show()


In [173]:
fig = px.density_heatmap(
    df_results,
    x="kernel_size",
    y="filters",
    z="best_val_f1",
    color_continuous_scale="Viridis",
    facet_col="pooling",
    title="Heatmap de Best Val F1"
)
fig.show()


**4. XARXES RECURRENTS**

A continuació afegirem capes recurrents al nostre model. En primer lloc, afegirem una capa *LSTM*. 

In [18]:
from keras.layers import LSTM

In [ ]:
# Importar métricas
from tensorflow.keras.metrics import F1Score

# Modelo
embedding_dim = 100
vocab_size = num_words + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))  # layer 1

# Afegim LSTM abans del pooling, i posem return_sequences=True perquè rebi 3D
model.add(LSTM(64, return_sequences=True)) 

model.add(GlobalMaxPooling1D())  # layer 2 (perdem temporalitat - podem provar sense)
model.add(Dense(128, activation='relu'))  # layer 3
model.add(Dense(num_classes, activation='softmax'))  # layer 4

metrics = tf.keras.metrics.F1Score(average='macro')
# Compile con métricas básicas
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = [metrics]
)

# Train
batch_size = 32
epochs = 20
model.fit(
    train_pad_sequences,
    train_encoded_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_pad_sequences, val_encoded_labels)
)

Epoch 1/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - f1_score: 0.0411 - loss: 1.5959 - val_f1_score: 0.0268 - val_loss: 2.3403
Epoch 2/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - f1_score: 0.0921 - loss: 0.6338 - val_f1_score: 0.0978 - val_loss: 1.8811
Epoch 3/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - f1_score: 0.1417 - loss: 0.4944 - val_f1_score: 0.1114 - val_loss: 1.7433
Epoch 4/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - f1_score: 0.1602 - loss: 0.3889 - val_f1_score: 0.1412 - val_loss: 1.4976
Epoch 5/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - f1_score: 0.1893 - loss: 0.3446 - val_f1_score: 0.1916 - val_loss: 1.2308
Epoch 6/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - f1_score: 0.2550 - loss: 0.2221 - val_f1_score: 0.2516 - val_loss: 1.1664
Epoch 7/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - f1_score: 0.3252 - loss: 0.1867 - val_f1_score: 0.2693 - val_loss: 0.8862
Epoch 8/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - f1_score: 0.3735 - loss: 0.1274 - val_f1_s

Provarem de modificar el valor de la LSTM per veure el seu resultat:

In [27]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))  # layer 1

# Afegim LSTM abans del pooling, i posem return_sequences=True perquè rebi 3D
model.add(LSTM(32, return_sequences=True)) 

model.add(GlobalMaxPooling1D())  # layer 2 (perdem temporalitat - podem provar sense)
model.add(Dense(128, activation='relu'))  # layer 3
model.add(Dense(num_classes, activation='softmax'))  # layer 4

metrics = tf.keras.metrics.F1Score(average='macro')
# Compile con métricas básicas
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = [metrics]
)

# Train
batch_size = 32
epochs = 20
model.fit(
    train_pad_sequences,
    train_encoded_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_pad_sequences, val_encoded_labels)
)

Epoch 1/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - f1_score: 0.0390 - loss: 1.8160 - val_f1_score: 0.0609 - val_loss: 2.3308
Epoch 2/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - f1_score: 0.1128 - loss: 0.6414 - val_f1_score: 0.1223 - val_loss: 1.4395
Epoch 3/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - f1_score: 0.1785 - loss: 0.3588 - val_f1_score: 0.2018 - val_loss: 1.0749
Epoch 4/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - f1_score: 0.2322 - loss: 0.2490 - val_f1_score: 0.2249 - val_loss: 0.9387
Epoch 5/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - f1_score: 0.3061 - loss: 0.1567 - val_f1_score: 0.2775 - val_loss: 0.7199
Epoch 6/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - f1_score: 0.3941 - loss: 0.1209 - val_f1_score: 0.3811 - val_loss: 0.7412
Epoch 7/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - f1_score: 0.4727 - loss: 0.1086 - val_f1_score: 0.4213 - val_loss: 0.6654
Epoch 8/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - f1_score: 0.5041 - loss: 0.0908 - val_f1_s

Apreciem que el nou valor de la mida del LSTM, que hem fet més petita, ha comportat una lleugera millora dels resultats del model.

Ara farem un experiment, provant d'entrenar els models amb diverses configuracions i estudiant el valor de LSTM que ens dona lloc a un millor resultat. 

Veiem que els resultats no són bons, i suposem d'entrada que pot ser degut a que el *GlobalMaxPooling1D* ens treu informació temporal de les seqüències (perdem l'ordre).

Així doncs, provarem d'entrenar el model sense la capa de *Pooling* per comparar-lo amb l'anterior. 

In [24]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))  # layer 1

model.add(LSTM(64)) 

model.add(Dense(256, activation='relu'))  # layer 3
model.add(Dense(num_classes, activation='softmax'))  # layer 4

metrics = tf.keras.metrics.F1Score(average='macro')
# Compile con métricas básicas
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = [metrics]
)

# Train
batch_size = 32
epochs = 10
model.fit(
    train_pad_sequences,
    train_encoded_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_pad_sequences, val_encoded_labels)
)

Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - f1_score: 0.0380 - loss: 1.5896 - val_f1_score: 0.0000e+00 - val_loss: 3.3597
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - f1_score: 0.0386 - loss: 1.1346 - val_f1_score: 0.0000e+00 - val_loss: 3.4058
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - f1_score: 0.0385 - loss: 1.1444 - val_f1_score: 0.0000e+00 - val_loss: 3.6584
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - f1_score: 0.0389 - loss: 1.0999 - val_f1_score: 0.0000e+00 - val_loss: 3.3659
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - f1_score: 0.0386 - loss: 1.1460 - val_f1_score: 0.0000e+00 - val_loss: 3.3982
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - f1_score: 0.0389 - loss: 1.0945 - val_f1_score: 0.0000e+00 - val_loss: 3.4122
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - f1_score: 0.0389 - loss: 1.1111 - val_f1_score: 0.0000e+00 - val_loss: 3.5705
Epoch 8/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - f1_score: 0.03

In [ ]:
# COMENTARIS DEL JORDI LUQUE
# (combinar convolucional i LSTM modela la seq. de correlacions i té sentit)
# amb pocs blocks transformer té sentit posar una LSTM
# Podem forçar mida de LSTM, + velocitat respecte heads de transformers
# Mirar distribució MITJANA de paraules en TRAIN per veure quins kernels fem servir
# Fem un pruning del vocabulary, NO fer servir el vocabulari sencer
# Potser provem HEADS i kernel 2 en convolucional

**5. REGULARITZACIÓ**

Per evitar l'overfitting del nostre model, hi afegirem *Dropout*.

**6. BALANCEJAT DE CLASSES**

Com que les classes del model estan molt desbalancejades, afegirem un pes a cada classe (class_weight) per afegir-lo al mètode *fit* del nostre model.

Calculem el pes de cada classe (per cadascuna de les *intencions* que tenim al model).

In [ ]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Entrenem el model de nou, aplicant els pesos de cada classe que hem calculat:

In [ ]:
# Train the model
batch_size = 32
epochs = 50
model.fit(train_pad_sequences, train_encoded_labels, class_weight=XXXXX, batch_size=batch_size, epochs=epochs, validation_data=(val_pad_sequences, val_encoded_labels))

Comprovem el nou resultat en el conjunt de *test*:

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_pad_sequences, test_encoded_labels, batch_size=batch_size)
print(f"Test accuracy: {accuracy[0]:.2f}")

Com podem observar, els resultats del nostre model han millorat molt respecte el model inicial.


<h1><a name="section-four"> 4. Lliurable </a></h1>

Heu d'entregar un document PDF de com a **màxim 10 pàgines** que incloga els resultats de tots els exercicis així com una explicació de cadascun dels resultats i de la modificació que heu fet. L'estructura del document és:

1. Introducció.
2. Experiments i Resultats (amb raonament).
3. Conclusions.

No cal que afegiu el vostre codi al document, podeu entregar el *notebook* juntament amb el document.

 ---